In [1]:
# 질의 임베딩 

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [3]:

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
# 'wiki' 인덱스를 가져옵니다.
index_name = "wine-review"
index = pc.Index(index_name)

index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wine': {'vector_count': 129971}},
 'total_vector_count': 129971,
 'vector_type': 'dense'}

In [5]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("winemag-data-130k-v2.csv")
docs = loader.load()

for i, doc in enumerate(docs[:3]):
    print(str(i), doc)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': 'winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
variety

In [8]:
from langchain_openai import OpenAIEmbeddings
import os
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    docs, 
    embedding, 
    index_name="wine-review", 
    namespace="wine"
)

KeyboardInterrupt: 

In [17]:
vector_store = PineconeVectorStore(
    index_name="wine-review",   # 기존에 만들어둔 인덱스 이름
    embedding=embedding,
    namespace="wine"            # 기존에 저장해둔 namespace
)

In [18]:
results = vector_store.similarity_search(
    "달콤한 맛을 느낄 수 있는 와인", 
    k=5, 
    namespace="wine"
)

results

[Document(id='a860bbe0-a652-49c2-89e1-e7691452e12f', metadata={'row': 47937.0, 'source': 'winemag-data-130k-v2.csv'}, page_content=': 47937\ncountry: US\ndescription: As sweet as honey, with Lifesaver candy pineapple, golden apricot preserves, Meyer lemon custard pie and Asian spice flavors. Calls for very rich fare, such as scallops sautéed in butter, served with a creamy risotto.\ndesignation: \npoints: 88\nprice: 22.0\nprovince: California\nregion_1: Sonoma County\nregion_2: Sonoma\ntaster_name: \ntaster_twitter_handle: \ntitle: Wellington 2007 Roussanne (Sonoma County)\nvariety: Roussanne\nwinery: Wellington'),
 Document(id='4cefa284-af69-4347-916f-8f7197331b50', metadata={'row': 45019.0, 'source': 'winemag-data-130k-v2.csv'}, page_content=": 45019\ncountry: US\ndescription: Once again this designate amazes with a surfeit of beautifully integrated flavors. Mushrooms and baking spices adorn an explosion of strawberry pastries. There's plenty of ripe cherry fruit also, and a long, po

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wine': {'vector_count': 130099}},
 'total_vector_count': 130099,
 'vector_type': 'dense'}